# Courseware Environment Build Manual
<HR>

## 1. Requirement of machine targeted for Courseware deployment
In order to build courseware, you need at least three machines below.
* Ansible : Use to build and deploy Courseware
* Hub : Management node of JupyterHub. Become an Access Point for built Courseware environment.
* Node : The node on which the Jupyter server runs.

But you can increase the _Node_ machine if necessary. (See section 3 for build directory) 

### Requirement
* **OS:** Ubuntu 14.04.x 
* **NIC:** 2 NICs  (For publick network and private network) 
* **Username:**  Each machines should be have an account with the user name "ubuntu".
* **SSH keys:**  It should also have the same public ssh key saved at /home/ubuntu/.ssh/authorized_keys and /root/.ssh/authorized_keys.The corresponding private key should be saved in the build directory in a file named **sshkey**.  (See section 3 for _build directory_) <br>
* **Please update:** The commands apt-get update and apt-get upgrade should be run on each instance.
   
### Example
* See <a href="#appendix_3">Appendix 3</a>.

## 2. Clone jupyter-platform-dev repository
Clone repository _jupyter-platform-dev_ of github  onto any machine.<br>
### Required packages
```
ssh-keygen
```
### Clone
```
$ cd YOUR_CLONE_DIR
$ git clone git@github.com:axsh/jupyter-platform-dev.git
```

## 3. Create the build directory
```
$ cd jupyter-platform-dev
$ nodecount=2 ./ind-steps/build-jh-environment/toplevel-generic-build.sh-new /some/directory/path/buildname
## ( The value for the environment variable nodecount could be 1, 2 or some other reasonable integer. )
```

## 4. Edit configuration files
The ```toplevel-generic-build.sh-new``` script creates a folder structure with the following files and contents:
```
$ head $(find /some/directory/path/buildname -name datadir.conf)
==> Node2 machine: /some/directory/path/buildname/jhvmdir-node2/datadir.conf <==
VMIP=192.168.999.999  # replace with the private IP used between instances
publicip=180.123.999.999 # replace with IP used by this script
publicport=22   # if needed, replace with the port used by this script

==> Node1 machine: /some/directory/path/buildname/jhvmdir-node1/datadir.conf <==
VMIP=192.168.999.999  # replace with the private IP used between instances
publicip=180.123.999.999 # replace with IP used by this script
publicport=22   # if needed, replace with the port used by this script

==> Ansible machine: /some/directory/path/buildname/jhvmdir/datadir.conf  <==
VMIP=192.168.999.999  # replace with the private IP used between instances
publicip=180.123.999.999 # replace with IP used by this script
publicport=22   # if needed, replace with the port used by this script

==> Hub machine: /some/directory/path/buildname/jhvmdir-hub/datadir.conf <== 
VMIP=192.168.999.999  # replace with the private IP used between instances
publicip=180.123.999.999 # replace with IP used by this script
publicport=22   # if needed, replace with the port used by this script

==> /some/directory/path/buildname/datadir.conf <==
node_list="node1 node2"
```
Each jhvmdir*/datadir.conf file should be edited to contain information for one instance. (In this example, there are 4, i.e. two docker swarm instances plus a hub instance, plus an instance for ansible.)

The _publicip_ variable value should be replaced by an IP address that can be used to ssh from the machine hosting the build directory to the corresponding instance. The publicport variable value should point to the ssh port, if port forwarding is used to reach the instance.

_VMIP_ should be a private IP address visible to all the other instances. Ssh must be possible to port 22 on this address.


### Example
* See <a href="#appendix_3">Appendix 3</a>.

## 5. Install Courseware
Once all the instances exist and all the information has been edited into the datadir.conf files, the following will install Courseware system, taking somewhat more than 60 minutes:
```
$ /path/to/just/a/little/disk/buildname/toplevel-generic-build.sh do
```

## 6. Check build
The build can be checked by running:
```
$ /path/to/just/a/little/disk/buildname/toplevel-generic-build.sh check
```

## 7. Inisialize course
$ ./bin/coursectl new COURSE_DIR MAIL_ADDRESS ACCESS_POINT
```
COURSE_DIR  : directory of the Courseware environmet each course. 
MAIL_ADDRESS : e-mail address of the course administrator (generally teacher of lecture).
ACCESS_POINT: access point of the course.  The FQDN of the Hub machine. 
```

```
ex)
$ cd YOUR_CLONE_DIR/jupyter-platform-dev
$ ./bin/coursectl new /some/directory/path/buildname foo@example.com hub.example.com
```

After execution, the login password of course administrator is displayed in the following format.
```
admin password: nQ8nv]E3wz
```
The course administrator can log in to the course with this email address and this password.<br>
You can change this password by _coursectl_ command. See <a href="#appendix_1">Appendix 1</a>.

## 8. Access course
You can access the built course by the following URL. *ACCESS_POINT* is the FQDN of the Hub machine.

https://ACCESS_POINT/login
```
Email Address: e-mail address of the course administrator
Password: Password returned by "/ bin / coursectl new"
```

<HR>
<a name="appendix_1"></a>
# Appendix 1 ： Change/Reset password
## Change password of the specified local user.
``` $ YOUR_CLONE_DIR/bin/coursectl change-password COURSE_DIR USER_EMAIL NEW_PASSWORD```
```
COURSE_DIR  : directory of the Courseware environmet each course.
USER_EMAIL : e-mail address of the local user to change the password.
NEW_PASSWORD : new password. 
```
## Rset password of the specified local user.
``` $ YOUR_CLONE_DIR/bin/coursectl reset-password COURSE_DIR USER_EMAIL```
```
COURSE_DIR  : directory of the Courseware environmet each course.
USER_EMAIL : e-mail address of the local user to change the password.

output : A new password will be displayed.
```


# Appendix 2 ： ssh wrapper scripts
If necessary, the ssh wrapper scripts for each instance can be modified directly. Normally this should only be necessary if special ssh parameters or workarounds are required.
```
$ cd /some/directory/path/buildname
$ find -name ssh-shortcut.sh
jhvmdir/ssh-shortcut.sh
jhvmdir-hub/ssh-shortcut.sh
jhvmdir-node3/ssh-shortcut.sh
jhvmdir-node2/ssh-shortcut.sh
jhvmdir-node1/ssh-shortcut.sh
```

<a name="appendix_3"></a>
# Appendix 3 ： Example for KVM set up
## Sample script for KVM set up 
This script will start 3 KVMs to set up an environment with one node.
```
#/!bin/bash

set -x

generate_setup_script() {
    addr="$1" hostname="$2"
cat <<EOF

echo 'ubuntu ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers
rm /etc/update-motd.d/*

# the next line is necessary or docker pulls do not work reliably
# related: https://bugs.debian.org/cgi-bin/bugreport.cgi?bug=625689
echo "prepend domain-name-servers 8.8.8.8;" | sudo tee -a /etc/dhcp/dhclient.conf

sudo apt-get update
sudo apt-get -y install git

sudo tee -a /etc/network/interfaces <<EOF2

auto eth1
iface eth1 inet static
    address $addr
    netmask 255.255.255.0
EOF2

# sudo ifdown eth1
sudo ifup eth1

echo "$hostname" >/etc/hostname
echo 127.0.0.1 "$hostname" >>/etc/hosts
hostname "$hostname"

EOF
}

ssh root@localhost -p 11022 -i ./sshkey <<<"$(generate_setup_script 192.168.44.10 ansible)"
ssh root@localhost -p 11122 -i ./sshkey <<<"$(generate_setup_script 192.168.44.11 hub)"
ssh root@localhost -p 11222 -i ./sshkey <<<"$(generate_setup_script 192.168.44.12 node1)"
```

### Example of datadir.conf for the set-up environment by the sample script
```
$ cat /some/directory/path/buildname/jhvmdir/datadir.conf
VMIP=192.168.44.10
publicip=127.0.0.1
publicport=11022

$ cat /some/directory/path/buildname/jhvmdir-hub/datadir.conf
VMIP=192.168.44.11
publicip=127.0.0.1
publicport=11122

$ cat /some/directory/path/buildname/jhvmdir-node1/datadir.conf
VMIP=192.168.44.12
publicip=127.0.0.1
publicport=11222
```